In [1]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

In [2]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_13.xlsx'))

In [3]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)


In [4]:
# Limpeza de linhas varias (NaN)
listagem = listagem.dropna()

In [5]:
#Criacao das colunas: codigo do tipo, id do projeto, data de proposicao do projeto, status da tramitacao, conferência da situacao
listagem['Cod_Tipo'] = ''
listagem['Id_Projeto'] = ''
listagem['Data_Projeto'] = ''
listagem['Tramitacao_Encerrada'] = ''
listagem['Situacao_Conf'] = '' 

In [6]:
# Preenchimento do campo Cod_Tipo
for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Cod_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Cod_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Cod_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Cod_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Cod_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Cod_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Cod_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Cod_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Cod_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Cod_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Cod_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Cod_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Cod_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Cod_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Cod_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Cod_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Cod_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Cod_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Cod_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Cod_Tipo'][i] = 'RCPI'
    else:
        listagem['Cod_Tipo'][i] = 'DESCONHECIDO'

In [7]:
# Preenchimento do campo Id_Projeto
count = 1
failed_req= 0
failed = False
id_projeto = ''
max_failed_req = 5

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '': 
            id_projeto = ''
            failed = False
            # Variaveis de requisicao
            num_proj = int(listagem['Projeto'][i])
            ano_proj = int(listagem['Ano'][i])
            tipo_proj = listagem['Cod_Tipo'][i]

            try:
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "tipo": tipo_proj}
                #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                except:
                    id_projeto = ''
            except:
                #Assina que houve falha
                failed = True
                    
            listagem['Id_Projeto'][i] = id_projeto
            time.sleep(0.5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break

    if failed_req > max_failed_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print('Fim do processo')

Fim do processo


In [ ]:
# Preenchimento do campo Data_Projeto
count = 1
failed_req= 0
failed = False
data_projeto = ''
max_failed_req = 5

while count > 0:
    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '': 
            data_projeto = ''
            failed = False
            # Variaveis de requisicao
            id_proj = listagem['Id_Projeto'][i]
            
            try:
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                                
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
            except:
                failed = True

            listagem['Data_Projeto'][i] = data_projeto
            time.sleep(0.5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print('Fim do processo')

In [9]:
# Preenchimento do campo Tramitacao_Encerrada
count = 1
failed_req= 0
failed = False
tramitacao_encerrada = ''
max_failed_req = 5

while count > 0:

    for i in listagem.index:

        if listagem['Tramitacao_Encerrada'][i] == '': 
            tramitacao_encerrada = ''
            failed = False
            
            try:
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto'][i]

                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                #query = {"numero": 44920}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)

                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                except:
                    tramitacao_encerrada = 'Informacao nao encontrada'
            except:
                failed = True

            listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada

            time.sleep(0.5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:

        if listagem['Tramitacao_Encerrada'][i] == '':
            count +=1


if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print('Fim do processo')

Fim do processo


In [ ]:
# Contagem e conferencia de qual eh o ultimo projeto com id_projeto preenchido 
count = 0
for i in listagem.index:
    
    if listagem['Id_Projeto'][i] == "":
        print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])
        print(count)
        break
    else:
        count +=1

In [ ]:
# Conferencia de valores
i = 936

print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], 
      listagem['Id_Projeto'][i], listagem['Data_Projeto'][i], listagem['Situação'][i])

#https://www.camarajf.mg.gov.br/sal/proposicao.php?num=45773

In [14]:
display(listagem)

,Projeto,Ano,Tipo,Autor,Ementa,Situação,Cod_Tipo,Id_Projeto,Data_Projeto,Tramitacao_Encerrada,Situacao_Conf
0,40520.0,2013.0,Requerimento,Oliveira Tresse,"Solicita urbanização na rua Antônio Celeste, n...",Arquivada,REQ,44205,,SIM,
2,4028.0,2013.0,Requerimento,Júlio Gasparette,Solicita limpeza nas bocas de lobo no bairro C...,Arquivada,REQ,44107,,SIM,
4,36840.0,2013.0,Requerimento,Luiz Otávio Fernandes Coelho - Pardal,solicita recapemamento asfáltico na rua Orland...,Em Tramitação,REQ,43978,,Informacao nao encontrada,
6,891.0,2013.0,Requerimento,Cido,Solicita o asfaltamento de toidas as ruas da C...,Arquivada,REQ,31106,,SIM,
8,8890.0,2013.0,Requerimento,Cido,Solicita o asfaltamento de todas as ruas da co...,Arquivada,REQ,27368,,SIM,
...,...,...,...,...,...,...,...,...,...,...,...
2578,4593.0,2013.0,Requerimento,João do Joaninho,"Solicita limpeza do córrego da rua Principal, ...",Arquivada,REQ,45320,,SIM,
2580,4592.0,2013.0,Requerimento,João do Joaninho,Solicita limpeza do córrego da rua Dona Euzébi...,Arquivada,REQ,45319,,SIM,
2582,4591.0,2013.0,Requerimento,João do Joaninho,Solicita reconstrução de uma tampa de bueiro n...,Arquivada,REQ,45318,,SIM,
2584,4590.0,2013.0,Requerimento,João do Joaninho,Solicita patrolamento das estradas de Rosário ...,Arquivada,REQ,45317,,SIM,


In [ ]:
# Exportacao da listagem para arquivo csv

path_to_file = '../ListagemExp/L_2010.csv'
path = Path(path_to_file)

if path.is_file():
    header = False
else:    
    header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=header) 
